In [120]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.isLeftEndpoint = False
    self.segmentId = None
  
  def __str__(self):
    return "(" + str(self.x) + ", " + str(self.y) + ")"

def printPointList(points):
  for p in points:
    print(p, end=" ")
  print()

def receiveInput():
    inputPoints = input().split()

    points = []

    for i in range(0, len(inputPoints) - 1, 2):
      points.append(Point(int(inputPoints[i]), int(inputPoints[i + 1])))

# points = receiveInput()

points = [(0, 3), (1, 2), (4, 4), (0, 0), (3, 1), (3, 3), (3.5, 2), (-2, 1)]
points = [Point(x, y) for x, y in points]

points2 = [(0, 8), (1, 7), (4, 9), (0, 5), (3, 6), (3, 8), (3.5, 7), (-2, 6)]
points2 = [Point(x, y) for x, y in points2]

In [121]:
def findBottomMostPoint(points):
  bottomMostPoint = points[0]

  for p in points:
    if p.y < bottomMostPoint.y or (p.y == bottomMostPoint.y and p.x < bottomMostPoint.x):
      bottomMostPoint = p

  return bottomMostPoint

In [122]:
from functools import cmp_to_key

def findOrientation(p0, p1, p2):
  crossProduct = (p1.x - p0.x) * (p2.y - p0.y) - (p2.x - p0.x) * (p1.y - p0.y)
  if crossProduct == 0:
    return "collinear"
  elif crossProduct > 0:
    return "right"
  else:
    return "left"

def findDistance(p0, p1):
  return ((p1.x - p0.x) ** 2 + (p1.y - p0.y) ** 2) ** 0.5

def sortPoints(points, bottomMostPoint):
  def comparePolarAngle(p1, p2):
    orientation = findOrientation(bottomMostPoint, p1, p2)
    if orientation == "collinear":
      if findDistance(bottomMostPoint, p1) <= findDistance(bottomMostPoint, p2):
        return -1
      else:
        return 1
    elif orientation == "right":
      return -1
    else:
      return 1
  sortedPoints = sorted(points, key = cmp_to_key(comparePolarAngle))
  return sortedPoints

def removeCollinearPoints(sortedPoints,bottomMostPoint):
  i = 0
  while i < len(sortedPoints) - 1:
    if findOrientation(bottomMostPoint, sortedPoints[i], sortedPoints[i + 1]) == "collinear":
      sortedPoints.pop(i)
    else:
      i += 1

In [123]:
def findConvexHull(points):
    bottomMostPoint = findBottomMostPoint(points)
    points.remove(bottomMostPoint)
    sortedPoints = sortPoints(points, bottomMostPoint)
    removeCollinearPoints(sortedPoints, bottomMostPoint)
    sortedPoints = [bottomMostPoint] + sortedPoints

    if len(sortedPoints) < 3:
      print("Convex hull is not possible!")

    convexHull = [sortedPoints[0], sortedPoints[1], sortedPoints[2]]
    for i in range(3, len(sortedPoints)):
        while findOrientation(convexHull[-2], convexHull[-1], sortedPoints[i]) != "right":
            convexHull.pop()
        convexHull.append(sortedPoints[i])
    return convexHull

In [124]:
convexHullA = findConvexHull(points)
convexHullB = findConvexHull(points2)
printPointList(convexHullA)
printPointList(convexHullB)


(0, 0) (3, 1) (3.5, 2) (4, 4) (0, 3) (-2, 1) 
(0, 5) (3, 6) (3.5, 7) (4, 9) (0, 8) (-2, 6) 

In [134]:
class Segment:
    def __init__(self, p1, p2):
        if p1.x < p2.x:
            self.p1 = p1
            self.p2 = p2
        else:
            self.p1 = p2
            self.p2 = p1
        # self.p1.isLeftEndpoint = True

    def __str__(self):
        return "(" + str(self.p1.x) + ", " + str(self.p1.y) + ", " + str(self.p1.isLeftEndpoint) + ") (" + str(self.p2.x) + ", " + str(self.p2.y) + ", " + str(self.p2.isLeftEndpoint)+ ")"

print(Segment(convexHullA[0], convexHullA[1]))


def printSegments(segments):
    print()
    for segment in segments:
        printPoint(segment.p1)
        print("-", end=" ")
        printPoint(segment.p2)
        print()

def findSegments(convexHull):
    segments = []
    for i in range(len(convexHull) - 1):
        segment = Segment(convexHull[i], convexHull[i + 1])
        segments.append(segment)
    segments.append(Segment(convexHull[-1], convexHull[0]))
    
    return segments

(0, 0, False) (3, 1, False)


In [126]:
printSegments(findSegments(convexHullA))


(0, 0) - (3, 1) 
(3, 1) - (3.5, 2) 
(3.5, 2) - (4, 4) 
(0, 3) - (4, 4) 
(-2, 1) - (0, 3) 
(-2, 1) - (0, 0) 


In [127]:
def onSegment(p1, p2, p3):
    if ((p3.x <= p2.x and p3.x >= p1.x) or (p3.x <= p1.x and p3.x >= p2.x)):
        if ((p3.y <= p2.y and p3.y >= p1.y) or (p3.y <= p1.y and p3.y >= p2.y)):
            return True
    return False

def segmentsIntersect(p1, p2, p3, p4):
    d1 = findOrientation(p3, p4, p1)
    d2 = findOrientation(p3, p4, p2)
    d3 = findOrientation(p1, p2, p3)
    d4 = findOrientation(p1, p2, p4)

    if ((d1 == 'right' and d2 == 'left') or (d1 == 'left' and d2 == 'right')) and ((d3 == 'right' and d4 == 'left') or (d3 == 'left' and d4 == 'right')): 
        return True
    elif d1 == 'collinear'and onSegment(p3,p4,p1):
        return True
    elif d2 == 'collinear'and onSegment(p3,p4,p2):
        return True
    elif d3 == 'collinear'and onSegment(p1,p2,p3):
        return True
    elif d4 == 'collinear'and onSegment(p1,p2,p4):
        return True
    else:
        return False


In [128]:
def anySegmentsIntersect(A_Hull, B_Hull):
    AHullSegments = findSegments(A_Hull)
    BHullSegments = findSegments(B_Hull)
    AHullSegments_size = len(AHullSegments)
    BHullSegments_size = len(BHullSegments)
    for i in range(AHullSegments_size):
        a1 = AHullSegments[i].p1
        a2 = AHullSegments[i].p2
        for f in range(BHullSegments_size):
            b1 = BHullSegments[f].p1
            b2 = BHullSegments[f].p2
            if (segmentsIntersect(a1, a2, b1, b2)):
                return True
    return False     
        

In [129]:
anySegmentsIntersect(convexHullA, convexHullB)

False

In [130]:
segmentsA = findSegments(convexHullA)
def findSegmentsEndpoints(segments):
    endpoints = []
    for segment in segments:
        leftEndpoint = segment.p1
        rightEndpoint = segment.p2

        endpoints.append(segment.p1)
        endpoints.append(segment.p2)
    return endpoints

def sortEndpoints(endpoints):
    def compareEndpoints(p1, p2):
        if p1.x < p2.x:
            return -1
        elif p1.x > p2.x:
            return 1
        else:
            if p1.isLeftEndpoint and not p2.isLeftEndpoint:
                return -1
            elif not p1.isLeftEndpoint and p2.isLeftEndpoint:
                return 1
            else:
                if p1.y < p2.y:
                    return -1
                elif p1.y > p2.y:
                    return 1
                else:
                    return 0
    sortedEndpoints = sorted(endpoints, key = cmp_to_key(compareEndpoints))
    return sortedEndpoints

segmentsAEndpoints = findSegmentsEndpoints(segmentsA)
sortedSegmentsAEndpoints = sortEndpoints(segmentsAEndpoints)
printPointList(segmentsAEndpoints)
printPointList(sortedSegmentsAEndpoints)


(0, 0) (3, 1) (3, 1) (3.5, 2) (3.5, 2) (4, 4) (0, 3) (4, 4) (-2, 1) (0, 3) (-2, 1) (0, 0) 
(-2, 1) (-2, 1) (0, 0) (0, 0) (0, 3) (0, 3) (3, 1) (3, 1) (3.5, 2) (3.5, 2) (4, 4) (4, 4) 

In [131]:
for i in segmentsAEndpoints:
    print("(", i.x, ", ", i.y, ") ", i.isLeftEndpoint, sep="")

(0, 0) False
(3, 1) False
(3, 1) False
(3.5, 2) False
(3.5, 2) False
(4, 4) False
(0, 3) False
(4, 4) False
(-2, 1) False
(0, 3) False
(-2, 1) False
(0, 0) False
